In [1]:
import weaviate
import json

import os
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index import VectorStoreIndex, ServiceContext
from llama_index import SimpleDirectoryReader
from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index.vector_stores import WeaviateVectorStore
from llama_index import VectorStoreIndex, StorageContext
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
from llama_index.response.notebook_utils import display_response

/home/kganapa/ResearchLLM/resllm/lib/python3.10/site-packages/llama_index/vector_stores/lancedb.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  from pandas import DataFrame


In [6]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    # model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path="/home/kganapa/ResearchLLM/Mistral_7b/mistral-7b-instruct-v0.2.Q4_K_M",
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=4096,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /home/kganapa/ResearchLLM/Mistral_7b/mistral-7b-instruct-v0.2.Q4_K_M (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loa

llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  16:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  17:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  18:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  19:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - kv  20:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  21:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  22:                    tokenizer.chat_template str              = {{ bos_token }}{% for message in mess...
llama_model_loader: - kv  23: 

In [7]:
documents = SimpleDirectoryReader(
    input_files = ["/home/kganapa/ResearchLLM/data/HGNN_General_Hypergraph_Neural_Networks.pdf"]
).load_data()

documents = Document(text = "\n\n".join([doc.text for doc in documents]))

In [8]:
client = weaviate.Client(
    url = "http://localhost:8080"
)

/home/kganapa/ResearchLLM/resllm/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


In [9]:
schema = {
            "classes": [
                {
                "class": "ArXivPaper",
                "description": "Represents a paper from ArXiv",
                "properties": [
                    {
                    "name": "title",
                    "dataType": ["string"],
                    "description": "Title of the paper"
                    },
                    {
                    "name": "authors",
                    "dataType": ["string"],
                    "description": "Authors of the paper"
                    },
                    {
                    "name": "abstract",
                    "dataType": ["string"],
                    "description": "Abstract of the paper"
                    },
                    {
                    "name": "content",
                    "dataType": ["text"],
                    "description": "Full text content of the paper"
                    },
                    {
                    "name": "url",
                    "dataType": ["string"],
                    "description": "URL of the paper on ArXiv"
                    },
                    {
                    "name": "published_date",
                    "dataType": ["date"],
                    "description": "Date when the paper was published on ArXiv"
                    }
                ]
                }
            ]
         }
client.schema.create(schema)

UnexpectedStatusCodeError: Create class! Unexpected status code: 422, with response body: {'error': [{'message': 'class name "ArXivPaper" already exists'}]}.

In [10]:
def get_build_index(documents,llm,embed_model="local:/home/kganapa/ResearchLLM/UAE-Large-V1",sentence_window_size=3,save_dir="./vector_store/index"):
  
  node_parser = SentenceWindowNodeParser(
      window_size = sentence_window_size,
      window_metadata_key = "window",
      original_text_metadata_key = "original_text"
  )

  vector_store = WeaviateVectorStore(weaviate_client = client, index_name="ArXivPaper", text_key="content")
  storage_context = StorageContext.from_defaults(vector_store = vector_store)


  sentence_context = ServiceContext.from_defaults(
      llm = llm,
      embed_model= embed_model,
      node_parser = node_parser,
  )

  if not os.path.exists(save_dir):
        # create and load the index
        index = VectorStoreIndex.from_documents(
            [documents], service_context=sentence_context, storage_context = storage_context
        )
        index.storage_context.persist(persist_dir=save_dir)
  else:
      # load the existing index
      index = load_index_from_storage(
          StorageContext.from_defaults(persist_dir=save_dir),
          service_context=sentence_context,
      )

  return index

In [11]:
# get the vector index
vector_index = get_build_index(documents=documents, llm=llm, embed_model="local:/home/kganapa/ResearchLLM/UAE-Large-V1", sentence_window_size=3, save_dir="./vector_store/index")

/home/kganapa/ResearchLLM/resllm/lib/python3.10/site-packages/weaviate/__init__.py:128: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Please import it from its specific module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
/home/kganapa/ResearchLLM/resllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def get_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):
  postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
  rerank = SentenceTransformerRerank(
      top_n=rerank_top_n, model="BAAI/bge-reranker-base"
  )
  engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
  )

  return engine

In [13]:
query_engine = get_query_engine(sentence_index=vector_index, similarity_top_k=6, rerank_top_n=2)

In [14]:
query="What are the hypergraph convolutions mentioned?"
response = query_engine.query(query)
display_response(response)
print("\n")


llama_print_timings:        load time =   37833.53 ms
llama_print_timings:      sample time =      34.06 ms /   145 runs   (    0.23 ms per token,  4256.57 tokens per second)
llama_print_timings: prompt eval time =   58363.21 ms /   682 tokens (   85.58 ms per token,    11.69 tokens per second)
llama_print_timings:        eval time =   13394.84 ms /   144 runs   (   93.02 ms per token,    10.75 tokens per second)
llama_print_timings:       total time =   72064.26 ms /   826 tokens


**`Final Response:`** The hypergraph convolutions mentioned in the context are two specific types of convolution layers used in Hypergraph Neural Networks (HGNNs). The first one is called HGNNConv, which is defined from the spectral aspect. The second one is called HGNNConv\_spatial, which is defined from the spatial aspect. The spectral aspect refers to the use of graph spectral theory to design the convolution operation, while the spatial aspect refers to the use of the spatial structure of the hypergraph to define the convolution operation. The text describes the spatial aspect of the hypergraph convolution in more detail, specifically a two-stage message passing framework for spatial hypergraph convolution.